In [1]:
!../data_processing/setup_spacy.sh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.5 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import sys; sys.path.append('../data_processing/')
import pandas as pd
from parse_dep import *
import re
from ast import literal_eval
import tokenizations
from datasets import load_dataset

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
bpe_pipeline = get_spacy_pipeline()

In [4]:
df = pd.read_csv('wiki_text_test_parsed2.pred.mwe', sep='\t', names=[0,'sentence','d'],)

In [5]:
wiki_text = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")['text']
wiki_text = [w for w in wiki_text if w]

Found cached dataset wikitext (/Users/andrejerkelens/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


In [6]:
df['d'] = df.apply(lambda x: literal_eval(x['d']), axis=1)

In [7]:
df = pd.concat([df, pd.json_normalize(df.d)],axis=1)
df = df.drop(columns=[0])

In [8]:
df['tokens'] = df['toks'].apply(lambda x: [y[0] for y in x])
df['sentence'] = df['sentence'].apply(lambda x: literal_eval(x).decode('utf-8'))

In [9]:
df.sentence[0]

'= Robert_Boulter ='

In [10]:
df.shape

(10265, 8)

---

### Note:

Only piece is to remove underscore from sentences 

In [11]:
nlp = spacy.load("en_core_web_sm")

In [99]:
def parse_sentences_mwe(dataset, pipeline = "spacy"):
    lines = []
    for d in dataset:
        if pipeline == "spacy":
            doc = nlp(d)
        elif pipeline == "bpe":
            doc = bpe_pipeline(d)
        if doc:
            for sent in doc.sents:
                lines.append(f"{word}\t{word.pos_}\n")
                lines.append("\n")
    return lines

def strip_g(l: list):
    return [i.replace("Ġ", "") for i in l]

In [100]:
_, two2one = tokenizations.get_alignments(df.iloc[1].tokens, list(map(str, list(bpe_pipeline(wiki_text[1]).sents)[0])))

In [101]:
df['sent'] = df['sentence'].apply(lambda x: x.replace("_", " ").replace("~", " "))

In [102]:
df.shape

(10265, 14)

In [103]:
df['bpe_tokens'] = df.apply(lambda x: list(map(str, list(bpe_pipeline(x["sent"])))), axis=1)

In [104]:
df['token_map'] = df.apply(lambda x: tokenizations.get_alignments(x['tokens'], 
                                                                  strip_g(x['bpe_tokens']))[1], axis=1)

In [106]:
df.iloc[52].token_map_dict

{0: [0],
 1: [1],
 2: [2],
 3: [3],
 5: [5],
 6: [6],
 7: [7],
 8: [8],
 9: [9],
 10: [10],
 11: [11],
 13: [17],
 14: [18, 19],
 16: [21],
 17: [22],
 18: [23],
 19: [24],
 20: [25],
 21: [26],
 22: [27],
 23: [28],
 24: [29],
 25: [30, 31],
 26: [31],
 27: [32]}

In [112]:
tokenizations.get_alignments(df.iloc[52].tokens, df.iloc[52].bpe_tokens)[1]

[[0],
 [1],
 [2],
 [3],
 [],
 [5],
 [6],
 [7],
 [8],
 [9],
 [10],
 [11],
 [],
 [],
 [],
 [],
 [],
 [13],
 [14],
 [14],
 [],
 [16],
 [17],
 [18],
 [19],
 [20],
 [21],
 [22],
 [23],
 [24],
 [25],
 [25, 26],
 [27]]

In [117]:
df.iloc[52].token_map_dict

{0: [0],
 1: [1],
 2: [2],
 3: [3],
 5: [5],
 6: [6],
 7: [7],
 8: [8],
 9: [9],
 10: [10],
 11: [11],
 13: [17],
 14: [18, 19],
 16: [21],
 17: [22],
 18: [23],
 19: [24],
 20: [25],
 21: [26],
 22: [27],
 23: [28],
 24: [29],
 25: [30],
 26: [31],
 27: [32]}

In [114]:
def list_to_index_dict(input_list):
    index_dict = {}
    
    for index, sublist in enumerate(input_list):
        for item in sublist:
            if item in index_dict:
                index_dict[item].append(index)
            else:
                index_dict[item] = [index]
    
    return index_dict

In [115]:
df['token_map_dict'] = df['token_map'].apply(lambda x: list_to_index_dict(x))

In [116]:
df[['sent','tags', '_', '~', 'tokens','bpe_tokens', 'token_map', 'token_map_dict']].iloc[52:53]

,sent,tags,_,~,tokens,bpe_tokens,token_map,token_map_dict
52,Du Fu ( Wade – Giles : Tu Fu ; Chinese : 杜甫 ; ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[[21, 22, 23], [26, 27]]",[],"[Du, Fu, (, Wade, –, Giles, :, Tu, Fu, ;, Chin...","[Du, ĠFu, Ġ(, ĠWade, ĠâĢĵ, ĠGiles, Ġ:, ĠTu, ĠF...","[[0], [1], [2], [3], [], [5], [6], [7], [8], [...","{0: [0], 1: [1], 2: [2], 3: [3], 5: [5], 6: [6..."


In [54]:
def map_weak_mwes_together(x):
    
    mapped_mwes = []
    for j, mwe in enumerate(x['_']):
        print("mwe")
        print(mwe)
        mapped_mwes.append([])
        for index in mwe:
            print("index")
            print(index)
            if index-1 in x['token_map_dict']:
                for val in x['token_map_dict'][index-1]:
                    print("val")
                    print(val)
                    mapped_mwes[j].append(val+1)
                
                
    return mapped_mwes
                   
    
def map_strong_mwes_together(x):
    
    mapped_mwes = []
    for j, mwe in enumerate(x['~']):
        mapped_mwes.append([])
        for index in mwe:
            
            if index-1 in x['token_map_dict']:
                for val in x['token_map_dict'][index-1]:
                    mapped_mwes[j].append(val+1)
                
                
    return mapped_mwes
                    

In [23]:
df['weak_mwe'] = df.apply(lambda x: map_weak_mwes_together(x),axis=1)
df['strong_mwe'] = df.apply(lambda x: map_strong_mwes_together(x),axis=1)

In [118]:
df.iloc[52].token_map_dict

{0: [0],
 1: [1],
 2: [2],
 3: [3],
 5: [5],
 6: [6],
 7: [7],
 8: [8],
 9: [9],
 10: [10],
 11: [11],
 13: [17],
 14: [18, 19],
 16: [21],
 17: [22],
 18: [23],
 19: [24],
 20: [25],
 21: [26],
 22: [27],
 23: [28],
 24: [29],
 25: [30],
 26: [31],
 27: [32]}

In [70]:
df.iloc[217:218]

,sentence,d,toks,tags,_,~,lemmas,tokens,sent,bpe_tokens,token_map,token_map_dict,weak_mwe,strong_mwe
217,The translators have had_to contend with bring...,"{'toks': [['The', 'DET'], ['translators', 'NOU...","[[The, DET], [translators, NOUN], [have, AUX],...","[O, O, O, B, Ī, O, O, B, Ī, O, O, O, O, O, O, ...","[[4, 5], [8, 9], [54, 55, 56], [57, 58]]",[],"[the, translator, have, have, to, contend, wit...","[The, translators, have, had, to, contend, wit...",The translators have had to contend with bring...,"[The, Ġtransl, ators, Ġhave, Ġhad, Ġto, Ġconte...","[[0], [1], [1], [2], [3], [4], [5], [6], [7], ...","{0: [0], 1: [1, 2], 2: [3], 3: [4], 4: [5], 5:...","[[5, 6], [9, 10, 10], [61, 62, 63], [64, 65]]",[]
